# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
# 获取含有噪音的数据集
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""

    # 括号中的 *，表示对list解开入参，即把列表元素分别当作参数传入
    dataset = data.TensorDataset(*data_arrays)

    # batch_size：表示的是小批量数据集的大小
    # shuffle=is_train：表示随机打乱顺序
    # 返回的是迭代器！！！！！！
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 1.5399,  0.7880],
         [ 0.7766,  1.2376],
         [ 0.1220,  0.7682],
         [ 0.3982, -0.3364],
         [ 1.5135, -1.3714],
         [ 0.9120,  1.4464],
         [-0.2449,  0.4629],
         [-0.0294, -0.8574],
         [ 1.0858, -1.0693],
         [ 0.2987,  0.4353]]),
 tensor([[ 4.6042],
         [ 1.5608],
         [ 1.8236],
         [ 6.1486],
         [11.8898],
         [ 1.1034],
         [ 2.1439],
         [ 7.0516],
         [10.0089],
         [ 3.3122]])]

使用框架的预定义好的层

In [3]:
# nn 是神经网络的意思
from torch import nn

# nn.Sequential(nn.Linear(2, 1))：nn.Linear(2, 1)表示的是输入的维度是 2，输出的维度是 1
# nn.Sequential()：指代的是一种容器
net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [4]:
# 这里的net[0]表示的是第一层layer
# 使用正态分布来替换掉 weight.data 的值
net[0].weight.data.normal_(0, 0.01)
# 直接填充数值 0 至 bias.data
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [5]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [6]:
# SGD：随机梯度下降
# SGD的核心思想是在每次迭代中随机选择一个样本（或一小批样本）来估计梯度，而不是使用整个数据集。
# 这样做的优点是计算效率高，尤其是当数据集很大时。
# SGD也能够逃离局部最小值，因为随机性引入了一定的噪声，有助于模型探索更多的参数空间。
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        # net(X)：表示的是将 X 进行传参
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    # l:f：表示的是 —— 打印 l，格式为浮点型
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000196
epoch 2, loss 0.000099
epoch 3, loss 0.000099


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [8]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0003, -0.0002])
b的估计误差： tensor([0.0003])
